In [4]:
import sympy as smp
from sympy.physics.mechanics import *
from sympy import symbols
from sympy import init_printing

In [5]:
phi,theta,x_b_l,x_b_r = dynamicsymbols('phi,theta,x_b_l,x_b_r')#机器人yaw旋转角度 机器人pitch倾斜角度 左滑块在轨道的位置 右滑块在轨道的位置
phi_d,theta_d,x_b_l_d,x_b_r_d = dynamicsymbols('phi,theta,x_b_l,x_b_r',1)#机器人旋转角速度 机器人倾斜角速度 左滑块在轨道的速度 右滑块在轨道的速度
phi_dd,theta_dd,x_b_l_dd,x_b_r_dd = dynamicsymbols('phi,theta,x_b_l,x_b_r',2)#机器人旋转角加速度 机器人倾斜角加速度 左滑块在轨道的加速度 右滑块在轨道的加速度
t = symbols('t')        #时间
q = dynamicsymbols('q') #机器人视为质点时的路程#机器人对坐标原点的位置矢量，随时间变化
u = smp.diff(q,t)       #机器人对坐标原点的速度矢量
u_d = smp.diff(u,t)     #机器人对坐标原点的加速度矢量
x_d = u*smp.cos(phi)    #机器人位置对坐标原点的速度矢量的x轴分量
y_d = u*smp.sin(phi)    #机器人位置对坐标原点的速度矢量的y轴分量
x = smp.integrate(x_d,t)#机器人位置对坐标原点位置的x轴分量
y = smp.integrate(y_d,t)#机器人位置对坐标原点位置的y轴分量
T_l,T_r,f_bl,f_br = dynamicsymbols('T_l,T_r,f_bl,f_br')#左轮电机输出力矩 右轮电机输出力矩 左滑块电机输出力 右滑块电机输出力
m_w,i_w,R,m,g,l,i_m,y_b,z_b,m_b = symbols('m_w,i_w,R,m,g,l,i_m,y_b,z_b,m_b')
#轮子质量 轮子转动惯量 轮子半径 车身质量 重力加速度 轮子质心到车体质心的距离 车身转动惯量 滑块相对车身坐标系y轴的轴向偏移位置 滑块相对轨道的高度 滑块质量
wheel_base = symbols('wheel_base')

In [6]:
world_0 = Point('world_0')
world_frame = ReferenceFrame('world')

cart_frame = world_frame.orientnew('cart','axis',[phi,world_frame.z])    # cart_frame: 车体旋转坐标系(yaw)

#机器人身体构造
base_0 = world_0.locatenew('base_0',x * world_frame.x + y * world_frame.y)#base_0: 机器人车体位置(视为质点)
base_0.set_vel(world_frame,x_d * world_frame.x + y_d * world_frame.y)    #机器人车体相对原点的运动速度
base_frame = cart_frame.orientnew('base','axis',[theta,cart_frame.y])    #base_frame: 机器人底盘坐标系/车体旋转坐标系(pitch)
base_mass_center = base_0.locatenew('base_mass_center',base_frame.z*l)   #base_inertia: 底盘质心
base_inertia = inertia(base_frame,0,i_m,0)    #底盘刚体：围绕y轴声明转动惯量
base_body = Body('base_body',base_mass_center,m,base_frame,base_inertia)

#机器人轮子构造
wheel_l_mc = base_0.locatenew('wheel_l_mc',wheel_base * base_frame.y)   #wheel_l: 左轮质心
wheel_r_mc = base_0.locatenew('wheel_r_mc',-wheel_base * base_frame.y)  #wheel_r: 右轮质心
wheel_l_mc.set_vel(world_frame,wheel_l_mc.pos_from(world_0).dt(world_frame))#左轮相对于原点的速度
wheel_r_mc.set_vel(world_frame,wheel_r_mc.pos_from(world_0).dt(world_frame))#右轮相对于原点的速度
wheel_l_frame = cart_frame.orientnew('wheel_l_frame','axis',[(q-phi*wheel_base/2)/R,cart_frame.y])#v*t=w*t*R -> s=θ*R,θ=s/R
wheel_r_frame = cart_frame.orientnew('wheel_r_frame','axis',[(q+phi*wheel_base/2)/R,cart_frame.y])#v*t=w*t*R -> s=θ*R,θ=s/R
wheel_l_frame.set_ang_vel(cart_frame,((u-phi_d*wheel_base/2)/R) * cart_frame.y)#v=w*R -> w=v/R
wheel_r_frame.set_ang_vel(cart_frame,((u+phi_d*wheel_base/2)/R) * cart_frame.y)
wheel_l_inertia = inertia(wheel_l_frame,0,i_w,0)
wheel_r_inertia = inertia(wheel_r_frame,0,i_w,0)
wheel_l_body = Body('wheel_l_body',wheel_l_mc,m_w,wheel_l_frame,wheel_l_inertia)
wheel_r_body = Body('wheel_r_body',wheel_r_mc,m_w,wheel_r_frame,wheel_r_inertia)

#机器人滑块构造
block_l_0 = base_0.locatenew('block_l_0',base_frame.x*x_b_l+base_frame.y*y_b+base_frame.z*(l+z_b))  # block: 动量块
block_r_0 = base_0.locatenew('block_r_0',base_frame.x*x_b_r-base_frame.y*y_b+base_frame.z*(l+z_b))
block_l_0.set_vel(world_frame,block_l_0.pos_from(world_0).dt(world_frame))
block_r_0.set_vel(world_frame,block_r_0.pos_from(world_0).dt(world_frame))
block_l_frame = base_frame.orientnew('block','axis',[0,base_frame.x])
block_r_frame = base_frame.orientnew('block','axis',[0,base_frame.x])
block_l_body = Body('block_l_body',block_l_0,m_b,block_l_frame)
block_r_body = Body('block_r_body',block_r_0,m_b,block_r_frame)

In [7]:
cross(-f_bl*base_frame.x,z_b*base_frame.z)

z_b*f_bl(t)*base.y

In [8]:
cross(-f_bl*base_frame.x,x_b_l*base_frame.x+z_b*base_frame.z+y_b*base_frame.y)

z_b*f_bl(t)*base.y - y_b*f_bl(t)*base.z

In [9]:
L = Lagrangian(world_frame,wheel_l_body,wheel_r_body,base_body,block_l_body,block_r_body)
#forecelist: 定义一些力和力矩，cross求的是力矩
LM = LagrangesMethod(L,[q,phi,theta,x_b_l,x_b_r],forcelist=[(base_mass_center,-m*g*world_frame.z),(block_l_0,-m_b*g*world_frame.z),(block_r_0,-m_b*g*world_frame.z),(wheel_l_frame,T_l*wheel_l_frame.y),(base_frame,-T_l*base_frame.y),(wheel_r_frame,T_r*wheel_r_frame.y),(base_frame,-T_r*base_frame.y),(base_frame,cross(f_bl*base_frame.x,x_b_l*base_frame.x+z_b*base_frame.z+y_b*base_frame.y)),(base_frame,cross(f_br*base_frame.x,z_b*base_frame.z-y_b*base_frame.y)),(block_l_0,f_bl*base_frame.x),(block_r_0,f_br*base_frame.x)],frame=world_frame)
laq_eqs = LM.form_lagranges_equations()

In [10]:
laq_eqs

Matrix([
[m*(l*(sin(phi(t))**2*cos(theta(t)) + cos(phi(t))**2*cos(theta(t)))*Derivative(theta(t), (t, 2)) + l*(-sin(phi(t))**2*sin(theta(t))*Derivative(theta(t), t) - sin(theta(t))*cos(phi(t))**2*Derivative(theta(t), t))*Derivative(theta(t), t) - (-l*sin(phi(t))*sin(theta(t))*Derivative(phi(t), t) + l*cos(phi(t))*cos(theta(t))*Derivative(theta(t), t))*sin(phi(t))*Derivative(phi(t), t) + (l*sin(phi(t))*cos(theta(t))*Derivative(theta(t), t) + l*sin(theta(t))*cos(phi(t))*Derivative(phi(t), t))*cos(phi(t))*Derivative(phi(t), t) + (-l*sin(phi(t))*sin(theta(t))*Derivative(phi(t), t)**2 - l*sin(phi(t))*sin(theta(t))*Derivative(theta(t), t)**2 + l*sin(phi(t))*cos(theta(t))*Derivative(theta(t), (t, 2)) + l*sin(theta(t))*cos(phi(t))*Derivative(phi(t), (t, 2)) + 2*l*cos(phi(t))*cos(theta(t))*Derivative(phi(t), t)*Derivative(theta(t), t))*sin(phi(t)) + (-l*sin(phi(t))*sin(theta(t))*Derivative(phi(t), (t, 2)) - 2*l*sin(phi(t))*cos(theta(t))*Derivative(phi(t), t)*Derivative(theta(t), t) - l*sin(thet

In [11]:
op_point = {q: 0, phi: 0, theta: 0, x_b_l: 0,x_b_r: 0,u: 0, phi_d: 0, theta_d: 0, x_b_l_d: 0, x_b_r_d:0,}
A, B, inp_vec = LM.linearize(q_ind=[q,phi,theta,x_b_l,x_b_r], qd_ind=[u,phi_d,theta_d,x_b_l_d,x_b_r_d], A_and_B=True, op_point=op_point)

In [13]:
B.evalf(subs={m_w:2.6,m:10.098,m_b:1.227,i_w:0.01683,l:0.039,y_b:0.16,z_b:0.0999,g:9.8,R:0.125,i_m:0.1639,wheel_base: 0.42})#B：状态向量X输入向量

Matrix([
[                 0,                  0,                  0,                  0],
[                 0,                  0,                  0,                  0],
[                 0,                  0,                  0,                  0],
[                 0,                  0,                  0,                  0],
[                 0,                  0,                  0,                  0],
[ 0.614753821325744,  0.614753821325744, 0.0132316980332364, 0.0132316980332364],
[ -1.65961661528494,   1.65961661528494,  0.158058725265232, -0.158058725265232],
[   -6.929097995272,    -6.929097995272, -0.586363305461782, -0.586363305461782],
[0.0821592317719468,  0.613236548663126,  0.908499486158217, 0.0429247690529679],
[ 0.613236548663126, 0.0821592317719468, 0.0429247690529679,  0.908499486158217]])

In [14]:
A.evalf(subs={m_w:2.6,m:10.098,m_b:1.227,i_w:0.01683,l:0.039,y_b:0.16,z_b:0.0999,g:9.8,R:0.125,i_m:0.1639,wheel_base:0.42})#A：状态向量X输出向量

Matrix([
[0, 0,                 0,                 0,                 0, 1.0,   0,   0,   0,   0],
[0, 0,                 0,                 0,                 0,   0, 1.0,   0,   0,   0],
[0, 0,                 0,                 0,                 0,   0,   0, 1.0,   0,   0],
[0, 0,                 0,                 0,                 0,   0,   0,   0, 1.0,   0],
[0, 0,                 0,                 0,                 0,   0,   0,   0,   0, 1.0],
[0, 0, -1.96106204011122, -1.59265141311766, -1.59265141311766,   0,   0,   0,   0,   0],
[0, 0,           0.e-140,           0.e-140,           0.e-140,   0,   0,   0,   0,   0],
[0, 0,  25.8383873397387,  70.5784204490064,  70.5784204490064,   0,   0,   0,   0,   0],
[0, 0,  8.17211003862151, -8.21069118724933, -8.21069118724933,   0,   0,   0,   0,   0],
[0, 0,  8.17211003862151, -8.21069118724933, -8.21069118724933,   0,   0,   0,   0,   0]])

In [15]:
inp_vec

Matrix([
[ T_l(t)],
[ T_r(t)],
[f_bl(t)],
[f_br(t)]])